# カルマンフィルタ

## 1. カルマンフィルタの必要性

### 1.1 問題設定: ノイズを含む観測からの状態推定

ロボットやセンサシステムでは、以下のような状況が頻繁に発生する。

- **真の状態** $x_k$（位置、速度など）を知りたい
- しかし、**センサで得られる観測値** $z_k$ にはノイズが含まれる
- システムの**動き自体にも不確実性**（プロセスノイズ）が存在する

例：ロボットの位置推定
- ロボットが「1m移動する」という指令を出しても、実際には 0.98m や 1.02m など誤差を含む
- GPSやセンサで位置を観測しても、ノイズにより真の位置からずれた値が得られる

### 1.2 単純な方法の限界

**方法1: 観測値をそのまま使う**
$$\hat{x}_k = z_k$$

問題点：観測ノイズがそのまま推定値に反映され、推定が不安定になる

**方法2: 移動平均**
$$\hat{x}_k = \frac{1}{N} \sum_{i=k-N+1}^{k} z_i$$

問題点:
- 過去のデータに同じ重みを与えるため、最新の情報が軽視される
- システムの動きのモデル（運動方程式）を活用していない

**カルマンフィルタの利点:**
- システムの動き（状態方程式）と観測（観測方程式）の両方をモデル化
- 予測と観測を最適に融合し、**最小二乗誤差の意味で最適な推定**を実現
- 各ステップで推定の不確実性（誤差共分散）も計算

---

## 2. 線形カルマンフィルタ

### 2.1 状態空間モデル

カルマンフィルタは、以下の2つの方程式でシステムを記述する。

#### 状態方程式

$$x_k = F_k x_{k-1} + B_k u_k + w_k$$

ここで:
- $x_k$: 時刻 $k$ における状態ベクトル（例: 位置、速度）
- $F_k$: 状態遷移行列（システムの動きを記述）
- $B_k$: 制御入力行列
- $u_k$: 制御入力（例：ロボットへの移動指令）
- $w_k$: プロセスノイズ $w_k \sim \mathcal{N}(0, Q_k)$
  - $Q_k$: プロセスノイズの共分散行列

#### 観測方程式

$$z_k = H_k x_k + v_k$$

ここで:
- $z_k$: 時刻 $k$ における観測値
- $H_k$: 観測行列（状態から観測への変換）
- $v_k$: 観測ノイズ $v_k \sim \mathcal{N}(0, R_k)$
  - $R_k$: 観測ノイズの共分散行列

#### 1次元ロボットの例

直線上を移動するロボット:
- 状態: $x_k$ （位置のみ）
- 制御入力: $u_k$ （移動距離）

状態方程式:
$$x_k = x_{k-1} + u_k + w_k$$

観測方程式:
$$z_k = x_k + v_k$$

ここで:
- $F_k = 1, B_k = 1, H_k = 1$
- プロセスノイズ: $w_k \sim \mathcal{N}(0, Q)$ where $Q = \sigma_w^2$
- 観測ノイズ: $v_k \sim \mathcal{N}(0, R)$ where $R = \sigma_v^2$

---

### 2.2 予測ステップ

予測ステップでは、前の時刻の推定値と制御入力から、現在の状態を**予測**する。

#### 状態の予測

時刻 $k-1$ での推定値を $\hat{x}_{k-1}$ とする。簡略化のため、以降では $\hat{x}_{k|k-1}$ を $\hat{x}_k^-$、$\hat{x}_{k|k}$ を $\hat{x}_k$ と表記する。

状態方程式の期待値をとると:

$$
\begin{align}
\hat{x}_k^- &= \mathbb{E}[x_k | z_{1:k-1}] \\
&= \mathbb{E}[F_k x_{k-1} + B_k u_k + w_k] \\
&= F_k \mathbb{E}[x_{k-1}] + B_k u_k + \mathbb{E}[w_k] \\
&= F_k \hat{x}_{k-1} + B_k u_k
\end{align}
$$

ここで $\mathbb{E}[w_k] = 0$ を用いた。

**1次元ロボットの場合:**
$$\hat{x}_k^- = \hat{x}_{k-1} + u_k$$

#### 誤差共分散の予測

推定誤差を $e_k = x_k - \hat{x}_k$ と定義する。予測時の誤差共分散を $P_k^-$ とすると:

$$P_k^- = \mathbb{E}[e_k^- {e_k^-}^T]$$

導出:

$$
\begin{align}
e_k^- &= x_k - \hat{x}_k^- \\
&= (F_k x_{k-1} + B_k u_k + w_k) - (F_k \hat{x}_{k-1} + B_k u_k) \\
&= F_k (x_{k-1} - \hat{x}_{k-1}) + w_k \\
&= F_k e_{k-1} + w_k
\end{align}
$$

したがって:

$$
\begin{align}
P_k^- &= \mathbb{E}[(F_k e_{k-1} + w_k)(F_k e_{k-1} + w_k)^T] \\
&= \mathbb{E}[F_k e_{k-1} e_{k-1}^T F_k^T] + \mathbb{E}[w_k w_k^T] \\
&= F_k P_{k-1} F_k^T + Q_k
\end{align}
$$

ここで $\mathbb{E}[e_{k-1} w_k^T] = 0$ （誤差とプロセスノイズは独立）を用いた。

**1次元ロボットの場合：**
$$P_k^- = P_{k-1} + Q$$

---

### 2.3 更新ステップ

更新ステップでは、観測値 $z_k$ を用いて予測値を**修正**する。

#### カルマンゲインの導出

予測値と観測値を重み付き平均で融合する:

$$\hat{x}_k = \hat{x}_k^- + K_k (z_k - H_k \hat{x}_k^-)$$

ここで：
- $K_k$: カルマンゲイン（最適な重み）
- $z_k - H_k \hat{x}_k^-$: イノベーション（観測値と予測値の差）

**目標:** 更新後の誤差共分散 $P_k$ を最小化する $K_k$ を求める。

更新後の推定誤差:

$$
\begin{align}
e_k &= x_k - \hat{x}_k \\
&= x_k - \hat{x}_k^- - K_k (z_k - H_k \hat{x}_k^-) \\
&= e_k^- - K_k (H_k x_k + v_k - H_k \hat{x}_k^-) \\
&= e_k^- - K_k (H_k e_k^- + v_k) \\
&= (I - K_k H_k) e_k^- - K_k v_k
\end{align}
$$

誤差共分散:

$$
\begin{align}
P_k &= \mathbb{E}[e_k e_k^T] \\
&= (I - K_k H_k) P_k^- (I - K_k H_k)^T + K_k R_k K_k^T
\end{align}
$$

#### 最適なカルマンゲイン

$P_k$ のトレース（分散の和）を最小化するため、$K_k$ で偏微分して0とおく:

$$\frac{\partial \text{tr}(P_k)}{\partial K_k} = 0$$

計算を進めると:

$$K_k = P_k^- H_k^T (H_k P_k^- H_k^T + R_k)^{-1}$$

これを用いると、誤差共分散の更新式は簡単な形になる:

$$P_k = (I - K_k H_k) P_k^-$$

**1次元ロボットの場合:**

カルマンゲイン:
$$K = \frac{P^-}{P^- + R}$$

状態更新:
$$\hat{x} = \hat{x}^- + K (z - \hat{x}^-)$$

誤差共分散更新:
$$P = (1 - K) P^-$$

#### カルマンゲインの意味

$$K = \frac{P^-}{P^- + R}$$

- $P^- \gg R$（予測の不確実性が大きい）→ $K \approx 1$ → 観測値を信頼
- $P^- \ll R$（観測の不確実性が大きい）→ $K \approx 0$ → 予測値を信頼

つまり、カルマンゲインは予測と観測の信頼度に応じて、自動的に最適な重みを決定する。

---

### 2.4 線形カルマンフィルタのまとめ

#### 初期化
$$\hat{x}_0 = x_0, \quad P_0 = P_0$$

#### 繰り返し（$k = 1, 2, 3, \ldots$）

**予測ステップ:**
$$
\begin{align}
\hat{x}_k^- &= F_k \hat{x}_{k-1} + B_k u_k \\
P_k^- &= F_k P_{k-1} F_k^T + Q_k
\end{align}
$$

**更新ステップ:**
$$
\begin{align}
K_k &= P_k^- H_k^T (H_k P_k^- H_k^T + R_k)^{-1} \\
\hat{x}_k &= \hat{x}_k^- + K_k (z_k - H_k \hat{x}_k^-) \\
P_k &= (I - K_k H_k) P_k^-
\end{align}
$$

#### 1次元ロボットの場合（$F=1, B=1, H=1$）

**予測:**
$$
\begin{align}
\hat{x}^- &= \hat{x} + u \\
P^- &= P + Q
\end{align}
$$

**更新:**
$$
\begin{align}
K &= \frac{P^-}{P^- + R} \\
\hat{x} &= \hat{x}^- + K (z - \hat{x}^-) \\
P &= (1 - K) P^-
\end{align}
$$

---

## 3. 拡張カルマンフィルタ

### 3.1 非線形システムの課題

線形カルマンフィルタは、システムが線形である場合にのみ最適である。しかし、多くの実システムは非線形である。

#### 非線形システムの例: 2次元ロボット

平面上を移動するロボット:
- 状態: $\mathbf{x} = [x, y, \theta]^T$ （位置と姿勢角）
- 制御入力: $\mathbf{u} = [v, \omega]^T$ （並進速度と角速度）

**非線形の運動モデル:**

$$
\begin{bmatrix}
x_k \\
y_k \\
\theta_k
\end{bmatrix}
=
\begin{bmatrix}
x_{k-1} + v \cos(\theta_{k-1}) \Delta t \\
y_{k-1} + v \sin(\theta_{k-1}) \Delta t \\
\theta_{k-1} + \omega \Delta t
\end{bmatrix}
+
\mathbf{w}_k
$$

この式は $\cos(\theta)$, $\sin(\theta)$ を含むため、**非線形**である。

線形カルマンフィルタは状態方程式が線形であることを仮定しているため、このような非線形システムには直接適用できない。

### 3.2 線形化とヤコビアン

拡張カルマンフィルタ（EKF）は、非線形関数を現在の推定値まわりで1次近似（線形化）することで、カルマンフィルタを非線形システムに拡張する。

#### 非線形状態空間モデル

状態方程式:
$$\mathbf{x}_k = f(\mathbf{x}_{k-1}, \mathbf{u}_k) + \mathbf{w}_k$$

観測方程式:
$$\mathbf{z}_k = h(\mathbf{x}_k) + \mathbf{v}_k$$

ここで $f$ と $h$ は非線形関数である。

#### テイラー展開による線形化

推定値 $\hat{\mathbf{x}}_{k-1}$ まわりで $f$ をテイラー展開すると:

$$f(\mathbf{x}_{k-1}, \mathbf{u}_k) \approx f(\hat{\mathbf{x}}_{k-1}, \mathbf{u}_k) + F_k (\mathbf{x}_{k-1} - \hat{\mathbf{x}}_{k-1})$$

ここで $F_k$ は**ヤコビ行列**:

$$F_k = \left. \frac{\partial f}{\partial \mathbf{x}} \right|_{\mathbf{x} = \hat{\mathbf{x}}_{k-1}}$$

同様に、観測方程式も線形化する:

$$H_k = \left. \frac{\partial h}{\partial \mathbf{x}} \right|_{\mathbf{x} = \hat{\mathbf{x}}_k^-}$$

---

### 3.3 2次元ロボットのヤコビアン

#### 運動モデル

$$
f(\mathbf{x}, \mathbf{u}) = 
\begin{bmatrix}
x + v \cos(\theta) \Delta t \\
y + v \sin(\theta) \Delta t \\
\theta + \omega \Delta t
\end{bmatrix}
$$

#### ヤコビ行列 $F_k$

$$
F_k = \frac{\partial f}{\partial \mathbf{x}} = 
\begin{bmatrix}
\frac{\partial f_1}{\partial x} & \frac{\partial f_1}{\partial y} & \frac{\partial f_1}{\partial \theta} \\
\frac{\partial f_2}{\partial x} & \frac{\partial f_2}{\partial y} & \frac{\partial f_2}{\partial \theta} \\
\frac{\partial f_3}{\partial x} & \frac{\partial f_3}{\partial y} & \frac{\partial f_3}{\partial \theta}
\end{bmatrix}
$$

各要素を計算すると:

$$
\begin{align}
\frac{\partial f_1}{\partial x} &= 1, \quad \frac{\partial f_1}{\partial y} = 0, \quad \frac{\partial f_1}{\partial \theta} = -v \sin(\theta) \Delta t \\
\frac{\partial f_2}{\partial x} &= 0, \quad \frac{\partial f_2}{\partial y} = 1, \quad \frac{\partial f_2}{\partial \theta} = v \cos(\theta) \Delta t \\
\frac{\partial f_3}{\partial x} &= 0, \quad \frac{\partial f_3}{\partial y} = 0, \quad \frac{\partial f_3}{\partial \theta} = 1
\end{align}
$$

したがって:

$$
F_k = 
\begin{bmatrix}
1 & 0 & -v \sin(\theta) \Delta t \\
0 & 1 & v \cos(\theta) \Delta t \\
0 & 0 & 1
\end{bmatrix}
$$

ここで $\theta$ は前ステップの推定値 $\hat{\theta}_{k-1}$ を用いる。

#### 観測モデル

観測は状態をそのまま観測すると考えると:

$$h(\mathbf{x}) = \mathbf{x} = \begin{bmatrix} x \\ y \\ \theta \end{bmatrix}$$

したがって、観測のヤコビアンは:

$$H_k = I_3 = \begin{bmatrix} 1 & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & 1 \end{bmatrix}$$

---

### 3.4 拡張カルマンフィルタのまとめ

#### 初期化
$$\hat{\mathbf{x}}_0 = \mathbf{x}_0, \quad P_0 = P_0$$

#### 繰り返し（$k = 1, 2, 3, \ldots$）

**予測ステップ:**

1. 状態予測（非線形モデルを直接使用）:
$$\hat{\mathbf{x}}_k^- = f(\hat{\mathbf{x}}_{k-1}, \mathbf{u}_k)$$

2. ヤコビアン計算:
$$F_k = \left. \frac{\partial f}{\partial \mathbf{x}} \right|_{\mathbf{x} = \hat{\mathbf{x}}_{k-1}}$$

3. 誤差共分散予測:
$$P_k^- = F_k P_{k-1} F_k^T + Q_k$$

**更新ステップ:**

1. ヤコビアン計算:
$$H_k = \left. \frac{\partial h}{\partial \mathbf{x}} \right|_{\mathbf{x} = \hat{\mathbf{x}}_k^-}$$

2. カルマンゲイン:
$$K_k = P_k^- H_k^T (H_k P_k^- H_k^T + R_k)^{-1}$$

3. 状態更新:
$$\hat{\mathbf{x}}_k = \hat{\mathbf{x}}_k^- + K_k (\mathbf{z}_k - h(\hat{\mathbf{x}}_k^-))$$

4. 誤差共分散更新:
$$P_k = (I - K_k H_k) P_k^-$$

---

## 4. まとめ

### 線形カルマンフィルタ

- **適用対象**: 線形システム
- **特徴**: 最小二乗誤差の意味で最適な推定
- **予測ステップ**: $\hat{x}^- = F\hat{x} + Bu$, $P^- = FPF^T + Q$
- **更新ステップ**: $K = P^-H^T(HP^-H^T + R)^{-1}$, $\hat{x} = \hat{x}^- + K(z - H\hat{x}^-)$

### 拡張カルマンフィルタ

- **適用対象**: 非線形システム
- **特徴**: 現在の推定値まわりで線形化し、近似的に最適な推定
- **予測ステップ**: $\hat{\mathbf{x}}^- = f(\hat{\mathbf{x}}, \mathbf{u})$, $P^- = F_kPF_k^T + Q$ （$F_k$ はヤコビアン）
- **更新ステップ**: 線形カルマンフィルタと同様（$H_k$ はヤコビアン）

### カルマンフィルタの本質

カルマンフィルタは:
1. システムの動きモデル（状態方程式）から状態を**予測**
2. センサの観測値を用いて予測を**修正**
3. 予測と観測の不確実性（共分散）を考慮し、最適な重み（カルマンゲイン）で融合

という3つのステップを繰り返すことで、ノイズを含む観測から真の状態を推定する。

---

## 参考文献

1. 上田隆一, "確率ロボティクス", 講義資料, 2025. [GitHub](https://github.com/ryuichiueda/slides_marp/tree/master/prob_robotics_2025)
2. Liu Yang (scomup), "カルマンフィルタと拡張カルマンフィルタ", Qiita, 2016. [Link](https://qiita.com/scomup/items/a23938a9d9a17fafa3f2)